# Create CMAQ-Ready File from Shapefile

---
    author: Barron H. Henderson
    date: 2020-04-25
    updated: 2024-02-05
---

This Notebook uses geopandas and cmaqsatproc to create IOAPI-like files for CMAQ. Geopandas supports STRtree optimized searches (via pygeos) and projection conversions. This simplifies the process to basic steps:

1. Process shapefile
    * Read in native projection.
    * Filter for in CMAQ domain.
    * Optionally, custom extra processing.
2. Calculate area overlap.
    * Perform grid cell intersections with shapefile polygons.
    * Optionally, define a weighted value (e.g, fraction of population).
    * Aggregate results to grid cell level.
    * Find largest area contributor.
    * Calculate total cell overlap.
3. Output
    * Store results as variables.
    * Save as IOAPI-like file



In [ ]:
# Install Libraries, only needs to be run first time on each machine
#!python -m pip install -qq cmaqsatproc geopandas xarray netcdf4 pycno

In [ ]:
# Download an example shapefile
#!wget https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_us_state_500k.zip # use with STUSPS
#!wget http://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip # use with ISO_A3
#!wget https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip # us with iso_3166_2

In [ ]:
import os
import string
import warnings
warnings.simplefilter('ignore')

import cmaqsatproc as csp
import geopandas as gpd

__version__ = '1.0'

In [ ]:
# shppath : str
#     Path to a shapefile or zip file containing a shapefile
shppath = 'cb_2022_us_state_500k.zip'

# attrkey : str
#     Column to group shapes by e.g., STUSPS of census (AL, NC, etc)
attrkey = 'STUSPS'

# prefix : str
#     Variable names with be <prefix>_<attrvalue> (STUSPS_NC)
prefix = f'{attrkey}_'

# gdpath : str
#     Path to a GRIDDESC file (optional for EPA domains 1US1, 4US1, 12US1,
#     12US2, 36US3, 108NHEMI2, etc)
gdpath = None

# gdnam : str
#     Name of grid definition within gdpath (e.g., 12US1, 108NHEMI2)
gdnam = '12US1'

# outformat : str
#     Either NETCDF4_CLASSIC or NETCDF3_CLASSIC (bigger, more IAOPI compatible)
outformat = 'NETCDF4_CLASSIC'

# debug : bool
#     Plot results for understanding what happened.
debug = True

# outpath : str
#     Defines where the output will be stored.
rootname = os.path.splitext(os.path.basename(shppath))[0]
outpath = f'./{rootname}.{attrkey}.{gdnam}.IOAPI.nc'

if os.path.exists(outpath):
    print(f'WARNING: Running this will overwrite existing file {outpath}.')
else:
    print(f'INFO: Running this will create {outpath}.')

In [ ]:
gf = csp.open_griddesc(gdnam, gdpath=gdpath)
bcrs = 'EPSG:4269'
if gf.GDTYP == 6:
    # For polar stereographic, make sure the bounds are reasonable.
    bbox = (-181, -30, 181, 91)
else:
    # The default bounding box is the perimiter in WGS84 + 1 degree buffer
    # You may need to update 4326 to match your shapefile
    bbox = gf.csp.geodf.envelope.to_crs(bcrs).unary_union.envelope.buffer(1).bounds

In [ ]:
shpf = gpd.read_file(shppath, bbox=bbox)
if attrkey not in shpf.columns:
    print('WARNING: attrkey must be in or derived from columns:')
    print(sorted(shpf.columns))
if bcrs != shpf.crs:
    print(f'INFO: Bounding box in {bcrs} and shapefile in {shpf.crs}.')
    print(f'INFO: If both are lat/lon variants, this should not be a problem.')
    print(f'INFO: You can optionally set bcrs=\'{shpf.crs}\' and run again')

In [ ]:
# Reproject input as grid projection and add custom variable with
# basic name cleanup
dshpf = shpf.to_crs(gf.csp.geodf.crs)
attrdefn = f'{attrkey} removing spaces and special characters'
# Derive custom key from another existing key
dshpf['custom'] = prefix + dshpf[attrkey]
# Cleanup names using a few common rules
dshpf['custom'] = dshpf['custom'].str.upper().str.replace('-', '_')
dshpf['custom'] = dshpf['custom'].str.replace('~', '_').str.replace('.', '_')
dshpf['custom'] = dshpf['custom'].str.replace('_99', 'UNK')
# add your own additional cleanup
# dshpf['custom'] = dshpf['custom']...

# Warn about weird names
allowed_ascii = string.ascii_uppercase + string.digits + '_'
for cat in dshpf['custom']:
    if cat[:1] not in string.ascii_uppercase:
        print(f'WARNING: {cat} starts with non ascii')
    for c in cat:
        if c not in allowed_ascii:
            print(f'WARNING: {cat} has unallowed characters')
            break

# only required if calculating weighted values
dshpf['shape_area'] = dshpf.geometry.area

In [ ]:
print('Unique Labels:')
print(dshpf['custom'].unique())

In [ ]:
# Calculate variables for each unique attribute value for fractional
# area overlap, the dominant value
domkey = f'{prefix}DOM'
totkey = f'{prefix}TOT'
# Advanced Options:
#   By default, this utility fraction area coverage. However, you can change
#   it to return a weighted attribute. For example, with Census shapefiles,
#   you could return fractional land area (ALAND). This might be more useful
#   with a variable like population.
srckey = 'intersection_area' # intersection_area or weighted_value
wgtkey = 'population'

attr_area = gpd.overlay(dshpf, gf.csp.geodf.reset_index(), how='intersection')
attr_area['intersection_area'] = attr_area.geometry.area
if srckey == 'weighted_value':
    if wgtkey not in dshpf.columns:
        raise KeyError(f'Cannot find {wgtkey}; must be one of {dshpf.columns}')
    # Add a custom_value (e.g., population)
    attr_area['weight'] = attr_area['intersection_area'] / attr_area['shape_area']
    attr_area['weighted_value'] = attr_area['weight'] * attr_area[wgtkey]

attrkeys = sorted(attr_area['custom'].unique())
attr_area['custom_idx'] = attr_area['custom'].apply(lambda x: attrkeys.index(x))
attr_area_sum = attr_area.groupby(['custom', 'ROW', 'COL']).sum(numeric_only=True)

dom_area = attr_area.sort_values(
    by=srckey, ascending=False
).groupby(['ROW', 'COL']).agg(**{
    'custom': ('custom', 'first'), domkey: (f'custom_idx', 'first'),
    totkey: (srckey, 'sum')
})
dom_area_ds = dom_area.to_xarray()
catds = attr_area_sum[srckey].unstack('custom').fillna(0).to_xarray()

catds[totkey] = dom_area_ds[totkey]

In [ ]:
# Add all variables to gf
for vark, var in catds.data_vars.items():
    print(vark, end=',', flush=True)
    gf[vark] = var.astype('f')
    gf[vark].attrs.update(
        long_name=vark.ljust(16),
        var_desc=f'Fractional overlap of {vark} with {gdnam}'.ljust(80)[:80],
        unit='1'.ljust(16)
    )

# Set any missing values (i.e., no overla) to 0.
for vark in gf.data_vars:
    gf[vark] = gf[vark].fillna(0)

# Add dominant key variable with custom missing value.
gf[domkey] = dom_area_ds[domkey].fillna(-999).astype('i')
gf[domkey] = gf[domkey].fillna(-999).astype('i')
gf[domkey].attrs.update(
    long_name=domkey.ljust(16),
    var_desc=f'Dominant {attrkey} for {gdnam} cell'.ljust(80)[:80],
    units='1'.ljust(16),
    description=str({k: i for i, k in enumerate(attrkeys)})[:-1] + ", 'UNASSIGNED': -999}"
)

# Add IOAPI meta-data
igf = gf.expand_dims(TSTEP=1, LAY=1).csp.to_ioapi()
igf.attrs['FILEDESC'] = f"""title: {outpath}
author: shapefile2cmaq
description: {attrkey} fractional area coverage, total ({prefix}TOT) and dominant ({prefix}DOM)
inputs:
 - Shapefile: {shppath}
 - Attribute: {attrdefn}
 - GDNAM: {gdnam}
 - GDPATH: {gdpath}
file_version: 1.0
tool_version: {__version__}
""".ljust(80*60)[:60*80]

In [ ]:
if debug:
    import matplotlib.pyplot as plt
    fig, axx = plt.subplots(1, 2, figsize=(12, 4))
    igf[domkey].where(lambda x: x > -999).plot(ax=axx[0], cmap='nipy_spectral')
    igf[totkey].plot(ax=axx[1], cmap='YlOrRd')
    _ = igf.csp.cno.drawcountries(ax=axx)
    print('Dominant Index')
    print(str({v: k.replace(prefix, '') for k, v in name2idx.items()}))

In [ ]:
os.makedirs(os.path.dirname(outpath), exist_ok=True)
igf.to_netcdf(outpath, format=outformat)